In [1]:
from confluent_kafka.admin import AdminClient
bootstrap_servers = "192.168.50.110:19092"  # 定义Kafka broker的地址
admin_client = AdminClient({"bootstrap.servers": bootstrap_servers}) # 创建一个AdminClient对象
topics_list = admin_client.list_topics().topics  # 调用list_topics()方法获取主题列表
topics_list

{'dc_abnormal_data': TopicMetadata(dc_abnormal_data, 3 partitions),
 'dc_test_event_forward': TopicMetadata(dc_test_event_forward, 3 partitions),
 'rum_data': TopicMetadata(rum_data, 3 partitions),
 'dc_databuff_process': TopicMetadata(dc_databuff_process, 3 partitions),
 'dc_databuff_dfni_reqs': TopicMetadata(dc_databuff_dfni_reqs, 3 partitions),
 'dc_gw_cmdb': TopicMetadata(dc_gw_cmdb, 3 partitions),
 'dc_databuff_log': TopicMetadata(dc_databuff_log, 3 partitions),
 'dc_abnormal_control': TopicMetadata(dc_abnormal_control, 1 partitions),
 'dc_aggs_policy_event': TopicMetadata(dc_aggs_policy_event, 3 partitions),
 'dc_agg_event': TopicMetadata(dc_agg_event, 3 partitions),
 '__consumer_offsets': TopicMetadata(__consumer_offsets, 50 partitions),
 'dc_databuff_container': TopicMetadata(dc_databuff_container, 3 partitions),
 'dc_databuff_resource': TopicMetadata(dc_databuff_resource, 3 partitions),
 'dc_fault_event': TopicMetadata(dc_fault_event, 3 partitions),
 'dc_databuff_k8s': TopicMe

In [3]:
from confluent_kafka.admin import AdminClient, NewTopic
bootstrap_servers = "192.168.50.110:19092"  # 定义Kafka broker的地址
admin_client = AdminClient({"bootstrap.servers": bootstrap_servers}) # 创建一个AdminClient对象

# 定义要创建的主题名称和配置
topic_name = "my_topic"
num_partitions = 1
replication_factor = 1

# 创建一个NewTopic对象
new_topic = NewTopic(
    topic=topic_name,
    num_partitions=num_partitions,
    replication_factor=replication_factor
)

# 调用create_topics()方法创建主题
admin_client.create_topics([new_topic])


True

In [11]:
import json
config = {}
config['bootstrap_servers'] = "192.168.50.110:19092"
config['event_topic'] = 'test'
config['offset'] = 'earliest'
config['groupid'] = 'test01'
new_topic = {}
new_topic['topic_name'] = 'test'
new_topic['partitions'] = 1
new_topic['replication'] = 1
config['new_topic'] = new_topic
with open('./config.json', 'w') as f:
    json.dump(config,f)

In [7]:
from confluent_kafka.admin import AdminClient

with open('./config.json', 'r') as f:
    config = json.load(f)

class Topic():
    def __init__(self):
        self.admin_client = AdminClient({"bootstrap.servers": config['bootstrap_servers']}) # 创建一个AdminClient对象
    
    def get_topic(self):
        topics_list = self.admin_client.list_topics().topics  # 调用list_topics()方法获取主题列表
        return topics_list
    
    def create_topic(self):
        new_topic = NewTopic(topic = config['new_topic']['topic_name'], 
                             num_partitions =  config['new_topic']['partitions'], 
                             replication_factor = config['new_topic']['replication'])
        admin_client.create_topics([new_topic])
        
    def main(self):
        if config['new_topic']['topic_name'] in self.get_topic().keys():
            return True
        else:
            self.create_topic()
            print('创建 topic 成功')

In [ ]:
from confluent_kafka import Consumer
import json
with open('./config.json', 'r') as f:
    config = json.load(f)
    
class getdata():
    def __init__(self):
        self.c = Consumer({
            'bootstrap.servers':config['bootstrap_servers'], # kafka所在ip地址
            'group.id':config['groupid'],
            'enable.auto.commit':True, # 是否自动提交offset，设为True时，每隔一段时间就会提交一次offset
            'default.topic.config':{
                'auto.offset.reset':config['offset']
            }  
        })
    
    def main(self):
        self.c.subscribe([config['event_topic']]) # 为consumer分配分区
        # 没有break会不停的取数据
        while True:
            msg = self.c.poll(100)
            if msg is None:
                continue
            else:
                topicMsg = msg.topic()
                message = json.loads(msg.value())

In [ ]:
from confluent_kafka import Producer
import json
with open('./config.json', 'r') as f:
    config = json.load(f)
    
def delivery_report(err, msg):
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        pass
    
class inputdata():
    def __init__(self):
        self.publisher = Producer({'bootstrap.servers': config['bootstrap_servers']})
    
    def main(self):
        for i in range(10000000):
            data = {'timestamp': time.time(), 'nums': i}
            publisher.poll(1)
            publisher.produce(config['event_topic'], json.dumps(data).encode('utf-8'),partition = 1, callback=delivery_report, timestamp = int(data['timestamp'])*1000)
            publisher.flush()
inputdata().main()

In [9]:
import time
data = {'timestamp': time.time(), 'nums': 1}
json.dumps(data).encode('utf-8')

b'{"timestamp": 1686119047.8197303, "nums": 1}'

In [10]:
time.time()

1686119076.929917